In [1]:
#import the necessary libraries 
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import folium
import statsmodels.api as sm
import scipy.stats
import numpy as np
from math import sqrt
import statsmodels.formula.api as smf



flows = pd.read_csv("london_flows.csv")

In [2]:
flows.head(10)


,station_origin,station_destination,flows,population,jobs,distance
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097
1,Abbey Road,Beckton,1,599,442,8510.121774
2,Abbey Road,Blackwall,3,599,665,3775.448872
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220
4,Abbey Road,Canning Town,37,599,15428,2228.923167
5,Abbey Road,Crossharbour,1,599,1208,6686.475560
6,Abbey Road,Custom House,0,599,845,3824.855630
7,Abbey Road,Cutty Sark,2,599,1748,8503.898909
8,Abbey Road,Cyprus,7,599,850,6532.099618
9,Abbey Road,Devons Road,1,599,611,3958.324171


In [3]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error


flows = pd.read_csv("london_flows.csv")
print(flows.info())
print(flows.head())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61474 entries, 0 to 61473
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   station_origin       61474 non-null  object 
 1   station_destination  61474 non-null  object 
 2   flows                61474 non-null  int64  
 3   population           61474 non-null  int64  
 4   jobs                 61474 non-null  int64  
 5   distance             61474 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 2.8+ MB
None
  station_origin station_destination  flows  population   jobs     distance
0     Abbey Road   Bank and Monument      0         599  78549  8131.525097
1     Abbey Road             Beckton      1         599    442  8510.121774
2     Abbey Road           Blackwall      3         599    665  3775.448872
3     Abbey Road        Canary Wharf      1         599  58772  5086.514220
4     Abbey Road        Canning Town     37  

In [4]:
import pandas as pd




cdatasubmat = flows.pivot_table(index='station_origin', columns='station_destination', values='flows', aggfunc='sum')


cdatasubmat.fillna(0, inplace=True)


cdatasubmat['All'] = cdatasubmat.sum(axis=1)
cdatasubmat.loc['All'] = cdatasubmat.sum()

cdatasubmat



station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,599.0
Acton Central,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1224.0
Acton Town,0.0,0.0,0.0,3.0,17.0,0.0,35.0,0.0,0.0,11.0,...,77.0,3.0,6.0,9.0,0.0,0.0,0.0,0.0,0.0,3745.0
Aldgate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,...,0.0,0.0,4.0,8.0,0.0,0.0,0.0,0.0,0.0,2886.0
Aldgate East,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,...,24.0,0.0,0.0,12.0,0.0,1.0,0.0,1.0,0.0,3172.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,0.0,0.0,2.0,5.0,47.0,0.0,0.0,0.0,0.0,22.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4868.0
Woodgrange Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,530.0
Woodside Park,0.0,0.0,1.0,26.0,11.0,0.0,0.0,0.0,0.0,59.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3093.0


In [5]:
flows['log_dest_pop'] = np.log(flows['population'] + 0.001)

prodsim_formula_exp = 'flows ~ station_origin + log_dest_pop + distance -1' 

prodsim = smf.glm(formula = prodsim_formula_exp, data=flows, family=sm.families.Poisson()).fit()
print(prodsim.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                61474
Model:                            GLM   Df Residuals:                    61074
Model Family:                 Poisson   Df Model:                          399
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.6957e+06
Date:                Wed, 04 Sep 2024   Deviance:                   3.2191e+06
Time:                        16:58:42   Pearson chi2:                 7.14e+06
No. Iterations:                    26   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                                                  coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

In [6]:
O_i = pd.DataFrame(flows.groupby(["station_origin"])["flows"].agg(np.sum))
O_i.rename(columns={"flows":"O_i"}, inplace = True)
flows = flows.merge(O_i, on = "station_origin", how = "left" )

D_j = pd.DataFrame(flows.groupby(["station_destination"])["flows"].agg(np.sum))
D_j.rename(columns={"flows":"D_j"}, inplace = True)
flows = flows.merge(D_j, on = "station_destination", how = "left" )

/tmp/ipykernel_2673/3741157495.py:1: FutureWarning: The provided callable <function sum at 0x7fce58fd5760> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'sum' instead.
  O_i = pd.DataFrame(flows.groupby(["station_origin"])["flows"].agg(np.sum))
/tmp/ipykernel_2673/3741157495.py:5: FutureWarning: The provided callable <function sum at 0x7fce58fd5760> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'sum' instead.
  D_j = pd.DataFrame(flows.groupby(["station_destination"])["flows"].agg(np.sum))


In [7]:
#We can do this by pulling out the parameter values
coefs = pd.DataFrame(prodsim.params)
coefs.reset_index(inplace=True)
coefs.rename(columns = {0:"alpha_i", "index":"coef"}, inplace = True)
to_repl = ["(station_origin)", "\[", "\]"]
for x in to_repl:
    coefs["coef"] = coefs["coef"].str.replace(x, "",regex=True)
#then once you have done this you can join them back into the dataframes
flows = flows.merge(coefs, left_on="station_origin", right_on="coef", how = "left")
flows.drop(columns = ["coef"], inplace = True)


In [8]:
flows.head()

,station_origin,station_destination,flows,population,jobs,distance,log_dest_pop,O_i,D_j,alpha_i
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097,6.395263,599,78549,-0.269855
1,Abbey Road,Beckton,1,599,442,8510.121774,6.395263,599,442,-0.269855
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.395263,599,665,-0.269855
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395263,599,58772,-0.269855
4,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395263,599,15428,-0.269855


In [9]:
alpha_i = prodsim.params[0:399]
gamma = prodsim.params[399]
beta = -prodsim.params[400]

/tmp/ipykernel_2673/4239489167.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  gamma = prodsim.params[399]
/tmp/ipykernel_2673/4239489167.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = -prodsim.params[400]


In [10]:
alpha_i

station_origin[Abbey Road]         -0.269855
station_origin[Acton Central]       0.250050
station_origin[Acton Town]         -0.393851
station_origin[Aldgate]            -0.957896
station_origin[Aldgate East]       -0.955320
                                      ...   
station_origin[Wood Street]         0.619874
station_origin[Woodford]            0.437553
station_origin[Woodgrange Park]     0.725765
station_origin[Woodside Park]       0.396496
station_origin[Woolwich Arsenal]    1.180225
Length: 399, dtype: float64

In [11]:
gamma

0.6137191414924278

In [12]:
beta

0.00014893502256351174

In [13]:
flows["prodsimest3"] = np.exp(flows["alpha_i"]+gamma*flows["log_dest_pop"] 
                                 - beta*flows["distance"])
#or you could do it the easy way like we did last week with the fitted column (See previous practical)
flows.head(10)

,station_origin,station_destination,flows,population,jobs,distance,log_dest_pop,O_i,D_j,alpha_i,prodsimest3
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097,6.395263,599,78549,-0.269855,11.518697
1,Abbey Road,Beckton,1,599,442,8510.121774,6.395263,599,442,-0.269855,10.887172
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.395263,599,665,-0.269855,22.037463
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395263,599,58772,-0.269855,18.128410
4,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395263,599,15428,-0.269855,27.745556
5,Abbey Road,Crossharbour,1,599,1208,6686.475560,6.395263,599,1208,-0.269855,14.284714
6,Abbey Road,Custom House,0,599,845,3824.855630,6.395263,599,845,-0.269855,21.875898
7,Abbey Road,Cutty Sark,2,599,1748,8503.898909,6.395263,599,1748,-0.269855,10.897267
8,Abbey Road,Cyprus,7,599,850,6532.099618,6.395263,599,850,-0.269855,14.616953
9,Abbey Road,Devons Road,1,599,611,3958.324171,6.395263,599,611,-0.269855,21.445339


In [15]:
# define r^2 and RMSE calculations
def CalcRSquared(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2
    
    return R2

def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    res = (observed -estimated)**2
    RMSE = round(np.sqrt(res.mean()), 3)
    
    return RMSE

In [16]:
CalcRSquared(flows["flows"], flows["prodsimest3"])

0.13590670847413897

In [17]:
CalcRMSE(flows["flows"], flows["prodsimest3"])

122.163